In [ ]:
# How to visualise?
# https://pypi.org/project/nipet/
# https://www.frontiersin.org/articles/10.3389/fninf.2018.00064/full
# https://github.com/marcomusy/vedo
# https://stackoverflow.com/questions/56035562/3d-dicom-visualisation-in-python
# https://www.youtube.com/watch?v=Tc9ONZLBHP0

# Saliency map?
# https://medium.com/@zeh.henrique92/basic-exploration-and-visualization-of-lung-ct-scans-with-python-e86b8d665bb6

TODO:
    * https://towardsdatascience.com/deep-learning-with-magnetic-resonance-and-computed-tomography-images-e9f32273dcb5
    * https://nilearn.github.io/manipulating_images/input_output.html
    * https://nilearn.github.io/auto_examples/plot_nilearn_101.html
    * https://vincentblog.xyz/posts/medical-images-in-python-computed-tomography?fbclid=IwAR33Nmfoq_u0RcdaXUSEWaJ4PlCOedqHafZnKTnQOQQk5mUuLC9LPvTaziU
    * https://nilearn.github.io/building_blocks/manual_pipeline.html
    * https://pandas.pydata.org/pandas-docs/version/0.13.1/visualization.html#targeting-different-subplots

In [15]:
# Fix relative import for utils
import sys
sys.path.append('..')

In [17]:
from utils import dataloader
from utils import display as dplay
from utils.adni import ADNI

import os
import numpy as np
import pandas as pd
from IPython.display import display 
import matplotlib.pyplot as plt

base_dir = '../data'

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
adni = ADNI(base_dir)

# Load data

In [ ]:
adni.load(show_output=False)

## Display MetaData

In [ ]:
meta_df = adni.meta_to_df()

In [ ]:
dplay.display_all_pd_cols(meta_df)

# Display ImageFiles

In [ ]:
files_df = adni.files_to_df()

In [ ]:
dplay.display_all_pd_cols(files_df)

# Save Images to adni folder as categorised data

In [ ]:
df = adni.to_df(show_output=False)
adni.save_to_category(df)

# Analysis

## MetaData distribution

In [ ]:
def plot_meta_settings(rows=1, cols=2, figsize=(16,16)):
    plt.rcParams.update({'font.size': 15})
    fig, axes = plt.subplots(nrows=rows, ncols=cols, figsize=figsize)
    #fig.tight_layout() #pad=8.0
    fig.set_tight_layout(True)
    #ax.autoscale(enable=True) 
    return fig, axes

### Overview

In [ ]:
fig, axes = plot_meta_settings(rows=3)

#.plot(); axes[0,0].set_title('A')
df['subject.researchGroup'].value_counts().plot(kind='bar',
                                    ax=axes[0,0],
                                    title="subject.researchGroup", xlabel="Disorder", ylabel="Frequency")

df['subject.subjectIdentifier'].value_counts().plot(kind='bar',
                                    ax=axes[0,1],
                                    title="subject.subjectIdentifier", xlabel="subjectIdentifier", ylabel="Frequency")

df['subject.subjectSex'].value_counts().plot(kind='bar',
                                    ax=axes[1,0],
                                    title="subject.subjectSex", xlabel="subjectSex", ylabel="Frequency")

df.groupby('subject.subjectSex')['subject.study.subjectAge'].plot(kind='kde',
                                    ax=axes[1,1],
                                    title="subject.study.subjectAge", xlabel="subjectAge", ylabel="Frequency",legend=True)

df['subject.study.series.dateAcquired'].value_counts().plot(
                                    ax=axes[2,0],
                                    title="subject.study.series.dateAcquired", xlabel="dateAcquired", ylabel="Frequency")

df.groupby('subject.subjectSex')['subject.study.weightKg'].plot(kind='kde',
                                    ax=axes[2,1],
                                    title="subject.study.weightKg", xlabel="weightKg", ylabel="Frequency",legend=True)



plt.show()

### Data sizes

In [ ]:
fig, axes = plot_meta_settings(rows=2)
df['subject.study.imagingProtocol.protocolTerm.protocol_Number of Slices'].value_counts().plot(kind="bar",
                                    ax=axes[0,0],
                                    title="subject.study.imagingProtocol.protocolTerm.protocol_Number of Slices", xlabel="Slices", ylabel="Frequency")

df['subject.study.imagingProtocol.protocolTerm.protocol_Number of Columns'].value_counts().plot(kind="bar",
                                    ax=axes[0,1],
                                    title="subject.study.imagingProtocol.protocolTerm.protocol_Number of Columns", xlabel="Slices", ylabel="Frequency")

df['subject.study.imagingProtocol.protocolTerm.protocol_Number of Rows'].value_counts().plot(kind="bar",
                                    ax=axes[1,0],
                                    title="subject.study.imagingProtocol.protocolTerm.protocol_Number of Rows", xlabel="Slices", ylabel="Frequency")
plt.show()

### Scoring

In [ ]:
fig, axes = plot_meta_settings(rows=3)
df['subject.visit.assessment.component.assessmentScore_FAQTOTAL'].value_counts().sort_index().plot(
                                    ax=axes[0,0],
                                    title="Functional Activities Questionnaires (FAQTOTAL)", xlabel="Score", ylabel="Frequency")

df['subject.visit.assessment.component.assessmentScore_NPISCORE'].value_counts().sort_index().plot(
                                    ax=axes[0,1],
                                    title="assessmentScore_NPISCORE", xlabel="Score", ylabel="Frequency")

df['subject.visit.assessment.component.assessmentScore_CDGLOBAL'].value_counts().sort_index().plot(
                                    ax=axes[1,0],
                                    title="Clinical Dementia Rating Scale (CDGLOBAL)", xlabel="Score", ylabel="Frequency")

df['subject.visit.assessment.component.assessmentScore_GDTOTAL'].value_counts().sort_index().plot(
                                    ax=axes[1,1],
                                    title="assessmentScore_GDTOTAL", xlabel="Score", ylabel="Frequency")

df['subject.visit.assessment.component.assessmentScore_MMSCORE'].value_counts().sort_index().plot(
                                    ax=axes[2,0],
                                    title="Mini-Mental State Examination (MMSCORE)", xlabel="Score", ylabel="Frequency")

for condition in ['MCI', 'CN', 'AD']:
    values = df[df['subject.researchGroup'] == condition]['subject.visit.assessment.component.assessmentScore_MMSCORE'].value_counts()
    (values / values.sum()).sort_index().plot(
                                    ax=axes[2,1],
                                    title="MMSE Score per Condition", xlabel="Score", ylabel="Density", label=condition)

plt.legend()
plt.show()

## Visualise brain slices

### Create Image generator

In [ ]:
SKIP_LAYERS = 10
LIMIT_LAYERS = 70

In [ ]:
image_AD_generator = adni.get_images(list(adni.get_files(adni.path.category['AD'], adni.category_cols)))
image_CN_generator = adni.get_images(list(adni.load_files(adni.path.category['CN'], adni.category_cols)))
image_MCI_generator = adni.get_images(list(adni.load_files(adni.path.category['MCI'], adni.category_cols)))

In [ ]:
### Testing functions
from nilearn.plotting import view_img, plot_glass_brain, plot_anat, plot_epi

In [ ]:
all_image = adni.get_images()

In [ ]:
test_image = next(all_image)

In [ ]:
test_image.shape

In [ ]:
plot_anat(test_image, draw_cross=False, display_mode='z',cut_coords=20,annotate=False)

In [ ]:
images_AD_array = adni.to_array(list(image_AD_generator))
images_CN_array = adni.to_array(list(image_CN_generator))
images_MCI_array = adni.to_array(list(image_MCI_generator))

In [ ]:
images_AD = next(images_AD_array)[0]
images_CN = next(images_CN_array)[0]
images_MCI = next(images_CN_array)[0]

### Coronal plane (From top)

In [ ]:

image_AD_slices = [images_AD[layer,:,:] for layer in range(0,images_AD.shape[0],SKIP_LAYERS)]
dplay.display_advanced_plot(image_AD_slices)
plt.suptitle("Coronal plane - AD")  

In [ ]:

image_CN_slices = [images_CN[layer,:,:] for layer in range(0,images_CN.shape[0],SKIP_LAYERS)]
dplay.display_advanced_plot(image_CN_slices)
plt.suptitle("Coronal plane - CN")  

In [ ]:
image_MCI_slices = [images_MCI[layer,:,:] for layer in range(0,images_MCI.shape[0],SKIP_LAYERS)]
dplay.display_advanced_plot(image_MCI_slices)
plt.suptitle("Coronal plane - MCI")  

### Sagittal plane (From front)

In [ ]:
image_slices = [images_AD[:,layer,:] for layer in range(0,images_AD.shape[1], SKIP_LAYERS)]
dplay.display_advanced_plot(image_slices)
plt.suptitle("Sagittal plane")  

### Horisontal plane (from side)

In [ ]:
image_slices = [images_AD[:,:,layer] for layer in range(0,images_AD.shape[2], SKIP_LAYERS)]
dplay.display_advanced_plot(image_slices)
plt.suptitle("Horisonal plane")  

# Notes
* Dont take 60 min dynamic - Iter(Brain Mod

In [ ]:
# Can we see a difference between MCI, CN and AD?

In [ ]:
# How is the images before preprocess?

In [ ]:
# How is the images after preprocess?

In [ ]:
# Distribution? (Can be text as well)